### Comparing Two Approaches to Calculating the Rotational Symmetry Index (Sigma) 

This notebook uses the examples from the 1996 paper to compare the original implementation of the sigma calcluation with a new method that uses 3D structures.  Both methods need more testing. 

In [1]:
!pip install rdkit numpy mols2grid pandas 

In [6]:
import requests, os
def download_text_file(url):
    res = requests.get(url)
    filename = os.path.basename(url)
    with open(filename,"w") as ifs:
        ifs.write(res.text)
download_text_file("https://raw.githubusercontent.com/PatWalters/escher/main/sigma.py")

In [7]:
from rdkit import Chem
import numpy as np
import mols2grid
import pandas as pd
from sigma import SigmaCalculator

Read the input data into a Pandas dataframe

In [8]:
buff = """C(F)(Cl)(Br)(I)
C1CCCCC1
c1ccccc1
C(C)(C)(Br)(Cl)
Br[C@H]1C[C@@H](Br)C[C@@H](Br)C1
BrC1C[C@@H](Br)C[C@H](Br)C1
CC(C)(C)C(C(C)(C)C)C(C)(C)C
CC(C)(C)CC(C)(C)C
C1Cc2cccc3cccc1c23
O=C1c2ccccc2C(=O)c2ccccc12
C1C3CC2CC(CC1C2)C3
Clc1ccc(Cl)cc1
C1CCC2(C1)CCCC2
c1ccc2c(c1)c1ccccc1c1ccccc21
c1cc2ccccc2[nH]1"""
df = pd.DataFrame(buff.split("\n"),columns=["SMILES"])

Add an RDKit molecule column to the dataframe

In [9]:
df['mol'] = df.SMILES.apply(Chem.MolFromSmiles)

Instantiate a **SigmaCalculator** object

In [10]:
sigma_calculator = SigmaCalculator()

Add columns with calculated sigma values
* orig - sigma calculated using the original 1996 algorithm
* new - sigma calculated using the new 3D algorithm

In [12]:
df['orig'] = df.mol.apply(sigma_calculator.calc_sigma_original, normalize=True)
df['new'] = df.mol.apply(sigma_calculator.calc_sigma_new)

Display the calculated sigma values.  The top value int the table below is sigma calculated using the original 1996 algorithm.  The bottom number is the sigma value calculated using the new algorithm. 

In [13]:
mols2grid.display(df[['mol','orig','new']],mol_col="mol",subset=["img",'orig','new'],
                  selection=False,n_cols=4,n_rows=5)